In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [2]:
rb= pd.read_csv('input/rb/rb_loc_cleaned.csv')
rb.head()


,games,year,Name,Team,Week,Opponent,att,yds,td,tar,rec_yds,def_rk,h/a_1,surface_0,points
0,1,2012,Adrian Peterson,MIN,1,JAX,17,84,2,1,3,30,1.0,0.0,21.7
1,1,2012,Ahmad Bradshaw,NYG,1,DAL,17,78,1,2,15,16,1.0,0.0,17.3
2,1,2012,Alfred Morris,WAS,1,NO,28,96,2,0,0,29,0.0,0.0,21.6
3,1,2012,Andre Brown,NYG,1,DAL,0,0,0,0,0,16,1.0,0.0,0.0
4,1,2012,Anthony Allen,BAL,1,CIN,4,13,0,0,0,27,1.0,1.0,1.3


In [3]:
rb['year'].unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018], dtype=int64)

In [6]:
rb_18=rb[rb['year']==2018]
rb_18['Week'].unique()

array([1, 2, 3, 4, 5, 6, 7, 8], dtype=int64)

In [7]:
rb.sort_values(['Name','games'], inplace=True)
rb.head()

,games,year,Name,Team,Week,Opponent,att,yds,td,tar,rec_yds,def_rk,h/a_1,surface_0,points
11995,1,2018,Aaron Jones,GB,3,WAS,6,42,0,1,5,15,0.0,1.0,5.7
12107,2,2018,Aaron Jones,GB,4,BUF,11,65,1,1,17,18,1.0,1.0,15.2
12212,3,2018,Aaron Jones,GB,5,DET,7,40,0,3,19,29,0.0,0.0,7.9
12317,4,2018,Aaron Jones,GB,6,SF,8,41,0,1,0,22,0.0,1.0,4.1
12533,5,2018,Aaron Jones,GB,8,LAR,12,86,1,2,0,14,0.0,1.0,16.6


In [8]:
rb.reset_index(drop=True, inplace=True)
rb.tail()

,games,year,Name,Team,Week,Opponent,att,yds,td,tar,rec_yds,def_rk,h/a_1,surface_0,points
12607,24,2017,Zurlon Tipton,IND,13,WAS,0,0,0,0,0,28,0.0,1.0,0.0
12608,25,2017,Zurlon Tipton,IND,14,CLE,0,0,0,0,0,13,0.0,1.0,0.0
12609,26,2017,Zurlon Tipton,IND,15,HOU,0,0,0,0,0,7,1.0,0.0,0.0
12610,27,2017,Zurlon Tipton,IND,16,DAL,1,-3,0,4,23,8,0.0,0.0,8.0
12611,28,2017,Zurlon Tipton,IND,17,TEN,9,21,0,2,45,12,1.0,0.0,6.6


In [9]:
rb.shape

(12612, 15)

In [10]:
def create_sorted_dict (df):
    df_sorted = df.sort_values(['Name','games'])
    sorted_dict = {}
    for player in df_sorted['Name']:
        sorted_dict['{}'.format(player)] = {'year':df_sorted[df_sorted['Name']==player]['year'],
                                            'Opponent':df_sorted[df_sorted['Name']==player]['Opponent'],
                                            'points':df_sorted[df_sorted['Name']==player]['points'],
                                            'def_rk':df_sorted[df_sorted['Name']==player]['def_rk'],
                                            'surface_0':df_sorted[df_sorted['Name']==player]['surface_0'],
                                            'h/a_1':df_sorted[df_sorted['Name']== player]['h/a_1'],
                                            'games': df_sorted[df_sorted['Name']==player]['games']
                                           }
       
    return sorted_dict

In [11]:
rb_sorted_dict = create_sorted_dict(rb)

In [140]:
# Create a data frame to investigate the points attribute of the dataset. 


def get_points_df(df):
    total_points = df['points'].sum()
    total_games = df['points'].count()

    val_lst = []
    sal_lst = []
    games_lst = []
    wght_avg=[]
    players = []
    avg_lst=[]

    for name in list(df['Name'].unique()):
        players.append(name)
        player_games = df[df['Name'] ==name].describe().loc['count'][0]
        games_lst.append(player_games)
        avg_points = round(df[df['Name'] ==name].describe()['points'].mean(),2)
        avg_lst.append(avg_points)
        wght_avg.append((round(avg_points + (avg_points* (player_games/total_games)),2)))
        sal_avg= round(df[df['Name'] ==name].describe()['salary'].mean(),2)
        sal_lst.append(sal_avg)
        val = round(sal_avg/avg_points,2)
        val_lst.append(val)
     
    df_eng = pd.DataFrame()
    df_eng['player'] = (df['Name'].unique())
    df_eng['games'] = games_lst
    df_eng['avg_points']= avg_lst
    df_eng['wght_avg'] = wght_avg
    df_eng['sal_avg'] = sal_lst
    df_eng['val'] = val_lst
    df_eng.head()
    
    return df_eng

## Group Dataframes

In [12]:
# A function to create a dictonary of grouped data frames

def grouped_dict(csv):
    
    df= pd.read_csv(csv)
    

    yearly_dict = {}
    for year in list(df['year'].unique()):
        yearly_dict['player_{}'.format(year)] = (df[df['year']==year]).drop('year',axis=1).groupby(['Name','games','Opponent']).sum()
        


    return yearly_dict

        
        

In [13]:
def conv_atts(grouped_dict):
    for group in grouped_dict:
        grouped_dict['{}'.format(group)]['h/a_1'] = grouped_dict['{}'.format(group)]['h/a_1'].astype('category')
        grouped_dict['{}'.format(group)]['surface_0'] = grouped_dict['{}'.format(group)]['surface_0'].astype('category')
    
    return grouped_dict

In [14]:
rb_group = grouped_dict('input/rb/rb_loc_cleaned.csv')

In [15]:
rb_group = conv_atts(rb_group)


In [16]:
rb_group['player_2013'].head()

Week  att  yds  td  tar  rec_yds  def_rk  \
Name            games Opponent                                             
Adrian Peterson 17    DET          1   18   93   2    4       18      10   
                18    CHI          2   26  100   0    2        7       4   
                19    CLE          3   25   88   1    6       27      26   
                20    PIT          4   23  140   2    1        0       1   
                21    CAR          6   10   62   0    4       21      29   

                               h/a_1 surface_0  points  
Name            games Opponent                          
Adrian Peterson 17    DET        0.0       0.0    29.1  
                18    CHI        0.0       1.0     8.7  
                19    CLE        1.0       0.0    15.5  
                20    PIT        1.0       0.0    26.0  
                21    CAR        1.0       0.0     8.3

In [17]:
rb_group

{'player_2012':                                 Week  att  yds  td  tar  rec_yds  def_rk  \
 Name            games Opponent                                             
 Adrian Peterson 1     JAX          1   17   84   2    1        3      30   
                 2     IND          2   16   60   0    3       20      21   
                 3     SF           3   25   86   0    4       21       8   
                 4     DET          4   21  102   0    4       20       5   
                 5     TEN          5   17   88   0    3       15      28   
                 6     WAS          6   17   79   0    8       50       4   
                 7     ARI          7   23  153   1    4        6       3   
                 8     TB           8   15  123   1    1        4      25   
                 9     SEA          9   17  182   2    4       11       6   
                 10    DET         10   27  171   1    4        5       5   
                 11    CHI         12   18  108   0    7     

In [22]:
def drop_col (grouped_df):
    for group in grouped_df:
        grouped_df[group].drop('h/a_1', axis=1, inplace=True)
        grouped_df[group].drop('surface_0',axis=1, inplace=True)
        grouped_df[group].drop('points',axis=1, inplace=True)
        grouped_df[group].drop('def_rk',axis=1, inplace=True)
        grouped_df[group].drop('Week', axis=1, inplace=True)

    return grouped_df

In [23]:
rb_group= drop_col(rb_group)

In [24]:
rb_group['player_2015'].columns

Index(['att', 'yds', 'td', 'tar', 'rec_yds'], dtype='object')

## Current Stats
Weekly moving average for every year from 2015 through 2018

In [25]:
# Week 1 stats are player's career average through 2014 season
# Populate current dataframe with current year averages of players

def insert_data (grouped_df):
    new_df = pd.DataFrame(index=grouped_df.index)
    for col in grouped_df: # Iterate through every attribute in the team_stats dataframe
        if col not in list(['def_rk','surface_0','h/a_1','games','points']):
            points=[]  # Create an empty list to hold the season average of the current attribute
            for player in grouped_df.index.levels[0]:  # Iterate through every player in the team_stats df
                total =[]# create an empty list to hold the weekly attributes valu3

                for week in grouped_df.loc[player].index: # Iterate throough every week for the current player
                    total.append(grouped_df.loc[player].loc[week][col]) # Add player's value for the current week's attribute to the total list
                    points.append(round(np.mean(total),2))  # Add the average of the total list to the season average list

            new_df['{}_avg'.format(col.lower())] = points # Insert season average list for current attribute into the current stats dataframe
        else:
            None
    return new_df

In [27]:
weekly_2012 = insert_data(rb_group['player_2012'])
weekly_2013 = insert_data(rb_group['player_2013'])
weekly_2014 = insert_data(rb_group['player_2014'])
weekly_2015 = insert_data(rb_group['player_2015'])
weekly_2016 = insert_data(rb_group['player_2016'])
weekly_2017 = insert_data(rb_group['player_2017'])
weekly_2018 = insert_data(rb_group['player_2018'])

In [28]:
weekly_2012.head()

att_avg  yds_avg  td_avg  tar_avg  rec_yds_avg
Name            games Opponent                                                
Adrian Peterson 1     JAX         17.00    84.00    2.00     1.00         3.00
                2     IND         16.50    72.00    1.00     2.00        11.50
                3     SF          19.33    76.67    0.67     2.67        14.67
                4     DET         19.75    83.00    0.50     3.00        16.00
                5     TEN         19.20    84.00    0.40     3.00        15.80

In [29]:
def shift_dict(df):

    shift_dict = {}
    df.reset_index(inplace=True)
    for player in df['Name'].unique():
        shift_dict[player]=df[df['Name']==player].shift(1).fillna(0)
    df_shifted = pd.concat(shift_dict)
    df_shifted.drop('Name', axis=1, inplace=True)
    
    
    
    return df_shifted

In [30]:
player_2012_shifted = shift_dict(weekly_2012)
player_2013_shifted = shift_dict(weekly_2013)
player_2014_shifted = shift_dict(weekly_2014)
player_2015_shifted = shift_dict(weekly_2015)
player_2016_shifted = shift_dict(weekly_2016)
player_2017_shifted = shift_dict(weekly_2017)
player_2018_shifted = shift_dict(weekly_2018)

In [31]:
player_2012_shifted['year']=2012
player_2013_shifted['year']=2013
player_2014_shifted['year']=2014
player_2015_shifted['year']=2015
player_2016_shifted['year']=2016
player_2017_shifted['year']=2017
player_2018_shifted['year']=2018

In [32]:
player_2018_shifted.head()

games Opponent  att_avg  yds_avg  td_avg  tar_avg  rec_yds_avg  \
Aaron Jones 0    0.0        0      0.0      0.0    0.00     0.00         0.00   
            1    1.0      WAS      6.0     42.0    0.00     1.00         5.00   
            2    2.0      BUF      8.5     53.5    0.50     1.00        11.00   
            3    3.0      DET      8.0     49.0    0.33     1.67        13.67   
            4    4.0       SF      8.0     47.0    0.25     1.50        10.25   

               year  
Aaron Jones 0  2018  
            1  2018  
            2  2018  
            3  2018  
            4  2018

In [33]:
player_weekly_avg = pd.concat([player_2012_shifted,player_2013_shifted, player_2014_shifted,player_2015_shifted, player_2016_shifted, player_2017_shifted, player_2018_shifted])

player_weekly_avg.head()

games Opponent  att_avg  yds_avg  td_avg  tar_avg  \
Adrian Peterson 0    0.0        0     0.00     0.00    0.00     0.00   
                1    1.0      JAX    17.00    84.00    2.00     1.00   
                2    2.0      IND    16.50    72.00    1.00     2.00   
                3    3.0       SF    19.33    76.67    0.67     2.67   
                4    4.0      DET    19.75    83.00    0.50     3.00   

                   rec_yds_avg  year  
Adrian Peterson 0         0.00  2012  
                1         3.00  2012  
                2        11.50  2012  
                3        14.67  2012  
                4        16.00  2012

In [34]:
player_weekly_avg.reset_index(inplace=True)
player_weekly_avg.head()

,level_0,level_1,games,Opponent,att_avg,yds_avg,td_avg,tar_avg,rec_yds_avg,year
0,Adrian Peterson,0,0.0,0,0.00,0.00,0.00,0.00,0.00,2012
1,Adrian Peterson,1,1.0,JAX,17.00,84.00,2.00,1.00,3.00,2012
2,Adrian Peterson,2,2.0,IND,16.50,72.00,1.00,2.00,11.50,2012
3,Adrian Peterson,3,3.0,SF,19.33,76.67,0.67,2.67,14.67,2012
4,Adrian Peterson,4,4.0,DET,19.75,83.00,0.50,3.00,16.00,2012


In [35]:
player_weekly_avg = player_weekly_avg.rename(columns={'level_0':'Name'}).drop('level_1', axis=1)
player_weekly_avg.head()

,Name,games,Opponent,att_avg,yds_avg,td_avg,tar_avg,rec_yds_avg,year
0,Adrian Peterson,0.0,0,0.00,0.00,0.00,0.00,0.00,2012
1,Adrian Peterson,1.0,JAX,17.00,84.00,2.00,1.00,3.00,2012
2,Adrian Peterson,2.0,IND,16.50,72.00,1.00,2.00,11.50,2012
3,Adrian Peterson,3.0,SF,19.33,76.67,0.67,2.67,14.67,2012
4,Adrian Peterson,4.0,DET,19.75,83.00,0.50,3.00,16.00,2012


In [36]:
player_weekly_avg.sort_values(['Name','year','games'],inplace=True)
player_weekly_avg.head()

,Name,games,Opponent,att_avg,yds_avg,td_avg,tar_avg,rec_yds_avg,year
11764,Aaron Jones,0.0,0,0.0,0.0,0.00,0.00,0.00,2018
11765,Aaron Jones,1.0,WAS,6.0,42.0,0.00,1.00,5.00,2018
11766,Aaron Jones,2.0,BUF,8.5,53.5,0.50,1.00,11.00,2018
11767,Aaron Jones,3.0,DET,8.0,49.0,0.33,1.67,13.67,2018
11768,Aaron Jones,4.0,SF,8.0,47.0,0.25,1.50,10.25,2018


In [37]:
player_weekly_avg.drop(['Name','games','Opponent'], axis=1, inplace=True)

In [38]:
player_weekly_avg.head()

,att_avg,yds_avg,td_avg,tar_avg,rec_yds_avg,year
11764,0.0,0.0,0.00,0.00,0.00,2018
11765,6.0,42.0,0.00,1.00,5.00,2018
11766,8.5,53.5,0.50,1.00,11.00,2018
11767,8.0,49.0,0.33,1.67,13.67,2018
11768,8.0,47.0,0.25,1.50,10.25,2018


In [39]:
player_weekly_avg.reset_index(inplace=True, drop=True)
player_weekly_avg.head()

,att_avg,yds_avg,td_avg,tar_avg,rec_yds_avg,year
0,0.0,0.0,0.00,0.00,0.00,2018
1,6.0,42.0,0.00,1.00,5.00,2018
2,8.5,53.5,0.50,1.00,11.00,2018
3,8.0,49.0,0.33,1.67,13.67,2018
4,8.0,47.0,0.25,1.50,10.25,2018


## Recent Player Stats
Calculate the 4 game moving average of all attributes for each player and insert resluts into positional dataframe

In [40]:
# A function designed to calcuate a player's 4 game moving average in seasons 2015,2016,2017,2018

def moving_average(grouped_df, games):
    new_grouped_df = pd.DataFrame(index = grouped_df.index)
    for col in grouped_df:  # Iterate through every attribute in the team_stats dataframe
        if col not in list(['def_rk','surface_0','h/a_1','games','points']):
            ma4 = []  # Create an empty list to hold the season average of the current attribute
            for player in grouped_df.index.levels[0]:  # Iterate through every player in the team_stats df
                total = [] # INstantiate a deque container with a max length of 3 to hold the three most current games for player
                count = 0 # Counter to track week number 
                for week in grouped_df.loc[player].index: # Iterate through every week for relevant player d
                    if count < games: # check count value to start 3 game moving average
                        ma4.append(0)  # week 1 through 3 have 3 game movong average of 0
                        total.append(grouped_df.loc[player].loc[week][col])  # Add the value current player's attribute total from the left
                        count += 1  # Increase count
                    else:  # Once 3 game moving average is avaliable
                        ma4.append(np.mean(total))  # Add the average of the 3 games held in totals 3 game moivng average list
                        total.append(grouped_df.loc[player].loc[week][col]) # Replace 3 set block with a first in first out process
        else:
            None
        new_grouped_df['{}_ma'.format(col.lower())] = ma4  # Populate databse with 3 game moving average list
       

    return new_grouped_df   

In [41]:
player_ma_2012 = moving_average(rb_group['player_2012'],4)
player_ma_2013 = moving_average(rb_group['player_2013'],4)
player_ma_2014 = moving_average(rb_group['player_2014'],4)
player_ma_2015 = moving_average(rb_group['player_2015'],4)
player_ma_2016 = moving_average(rb_group['player_2016'],4)
player_ma_2017 = moving_average(rb_group['player_2017'],4)
player_ma_2018 = moving_average(rb_group['player_2018'],4)

In [42]:
yearly_ma_grp = pd.concat([player_ma_2012, player_ma_2013])
yearly_ma_grp = pd.concat([yearly_ma_grp, player_ma_2014])
yearly_ma_grp = pd.concat([yearly_ma_grp, player_ma_2015])
yearly_ma_grp = pd.concat([yearly_ma_grp, player_ma_2016])
yearly_ma_grp = pd.concat([yearly_ma_grp,player_ma_2017])
yearly_ma_grp = pd.concat([yearly_ma_grp,player_ma_2018])
yearly_ma_grp.head()

att_ma  yds_ma  td_ma  tar_ma  rec_yds_ma
Name            games Opponent                                           
Adrian Peterson 1     JAX         0.00     0.0    0.0     0.0         0.0
                2     IND         0.00     0.0    0.0     0.0         0.0
                3     SF          0.00     0.0    0.0     0.0         0.0
                4     DET         0.00     0.0    0.0     0.0         0.0
                5     TEN        19.75    83.0    0.5     3.0        16.0

In [43]:
yearly_ma_grp.reset_index(inplace=True)
yearly_ma_grp.head()

,Name,games,Opponent,att_ma,yds_ma,td_ma,tar_ma,rec_yds_ma
0,Adrian Peterson,1,JAX,0.00,0.0,0.0,0.0,0.0
1,Adrian Peterson,2,IND,0.00,0.0,0.0,0.0,0.0
2,Adrian Peterson,3,SF,0.00,0.0,0.0,0.0,0.0
3,Adrian Peterson,4,DET,0.00,0.0,0.0,0.0,0.0
4,Adrian Peterson,5,TEN,19.75,83.0,0.5,3.0,16.0


In [81]:
yearly_ma_grp.drop(['def_rk_ma','surface_0_ma','h/a_1_ma','points_ma'],axis=1, inplace=True)

In [44]:
yearly_ma_grp.sort_values(['Name','games'], inplace=True)
yearly_ma_grp.head()

,Name,games,Opponent,att_ma,yds_ma,td_ma,tar_ma,rec_yds_ma
11764,Aaron Jones,1,WAS,0.0,0.0,0.00,0.0,0.00
11765,Aaron Jones,2,BUF,0.0,0.0,0.00,0.0,0.00
11766,Aaron Jones,3,DET,0.0,0.0,0.00,0.0,0.00
11767,Aaron Jones,4,SF,0.0,0.0,0.00,0.0,0.00
11768,Aaron Jones,5,LAR,8.0,47.0,0.25,1.5,10.25


In [45]:
yearly_ma_grp.reset_index(drop=True, inplace=True)
yearly_ma_grp.head()

,Name,games,Opponent,att_ma,yds_ma,td_ma,tar_ma,rec_yds_ma
0,Aaron Jones,1,WAS,0.0,0.0,0.00,0.0,0.00
1,Aaron Jones,2,BUF,0.0,0.0,0.00,0.0,0.00
2,Aaron Jones,3,DET,0.0,0.0,0.00,0.0,0.00
3,Aaron Jones,4,SF,0.0,0.0,0.00,0.0,0.00
4,Aaron Jones,5,LAR,8.0,47.0,0.25,1.5,10.25


In [46]:
player_weekly_avg.shape, yearly_ma_grp.shape

((12612, 6), (12612, 8))

In [47]:
temp= pd.concat([player_weekly_avg,yearly_ma_grp],axis=1)
temp.isnull().sum()

att_avg        0
yds_avg        0
td_avg         0
tar_avg        0
rec_yds_avg    0
year           0
Name           0
games          0
Opponent       0
att_ma         0
yds_ma         0
td_ma          0
tar_ma         0
rec_yds_ma     0
dtype: int64

In [48]:
temp.head()

,att_avg,yds_avg,td_avg,tar_avg,rec_yds_avg,year,Name,games,Opponent,att_ma,yds_ma,td_ma,tar_ma,rec_yds_ma
0,0.0,0.0,0.00,0.00,0.00,2018,Aaron Jones,1,WAS,0.0,0.0,0.00,0.0,0.00
1,6.0,42.0,0.00,1.00,5.00,2018,Aaron Jones,2,BUF,0.0,0.0,0.00,0.0,0.00
2,8.5,53.5,0.50,1.00,11.00,2018,Aaron Jones,3,DET,0.0,0.0,0.00,0.0,0.00
3,8.0,49.0,0.33,1.67,13.67,2018,Aaron Jones,4,SF,0.0,0.0,0.00,0.0,0.00
4,8.0,47.0,0.25,1.50,10.25,2018,Aaron Jones,5,LAR,8.0,47.0,0.25,1.5,10.25


In [49]:
temp.isnull().sum()

att_avg        0
yds_avg        0
td_avg         0
tar_avg        0
rec_yds_avg    0
year           0
Name           0
games          0
Opponent       0
att_ma         0
yds_ma         0
td_ma          0
tar_ma         0
rec_yds_ma     0
dtype: int64

In [50]:
cols = temp.columns.tolist()
cols = cols[5:9] + cols[:5] + cols[9:]
temp = temp[cols]
temp.head()

,year,Name,games,Opponent,att_avg,yds_avg,td_avg,tar_avg,rec_yds_avg,att_ma,yds_ma,td_ma,tar_ma,rec_yds_ma
0,2018,Aaron Jones,1,WAS,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00
1,2018,Aaron Jones,2,BUF,6.0,42.0,0.00,1.00,5.00,0.0,0.0,0.00,0.0,0.00
2,2018,Aaron Jones,3,DET,8.5,53.5,0.50,1.00,11.00,0.0,0.0,0.00,0.0,0.00
3,2018,Aaron Jones,4,SF,8.0,49.0,0.33,1.67,13.67,0.0,0.0,0.00,0.0,0.00
4,2018,Aaron Jones,5,LAR,8.0,47.0,0.25,1.50,10.25,8.0,47.0,0.25,1.5,10.25


## Career Average

In [51]:
# Week 1 stats are player's career average through 2014 season
# Populate current dataframe with current year averages of players

def career_average (grouped_df1, grouped_df2=None):
    new_grouped_df1 = pd.DataFrame(index= grouped_df1.index)
    for col in grouped_df1: # Iterate through every attribute in the team_stats dataframe
        points=[]  # Create an empty list to hold the season average of the current attribute

        try:
            for player in grouped_df1.index.levels[0]:  # Iterate through every player in the team_stats df
                if player in list(grouped_df2.index.levels[0].unique()):
                    total =[grouped_df2.loc[player][col][-1]] # If player not a rookie, use final average from 2015
                else:
                    total =[]  # create an empty list to hold the weekly attributes value
                
                for week in grouped_df1.loc[player].index: # Iterate throough every week for the current player
                    total.append(grouped_df1.loc[player].loc[week][col]) # Add player's value for the current week's attribute to the total list
                    points.append(round(np.mean(total),2))  # Add the average of the total list to the season average list
                    
            new_grouped_df1['{}_car'.format(col.lower())] = points # Insert season average list for current attribute into the current stats dataframe


                                
        except:
            for player in grouped_df1.index.levels[0]:  # Iterate through every player in the team_stats df
                total = []

                for week in grouped_df1.loc[player].index: # Iterate throough every week for the current player
                    total.append(grouped_df1.loc[player].loc[week][col]) # Add player's value for the current week's attribute to the total list
                    points.append(round(np.mean(total),2))  # Add the average of the total list to the season average list

            new_grouped_df1['{}_car'.format(col.lower())] = points # Insert season average list for current attribute into the current stats dataframe
    return new_grouped_df1

In [52]:
df_career_2012 = career_average(rb_group['player_2012'])
df_career_2013 = career_average(rb_group['player_2013'], rb_group['player_2012'])
df_career_2014 = career_average(rb_group['player_2014'], rb_group['player_2013'])
df_career_2015 = career_average(rb_group['player_2015'], rb_group['player_2014'])
df_career_2016 = career_average(rb_group['player_2016'], rb_group['player_2015'])
df_career_2017 = career_average(rb_group['player_2017'], rb_group['player_2016'])
df_career_2018 = career_average(rb_group['player_2018'], rb_group['player_2017'])

In [53]:
player_career_grp = pd.concat([df_career_2012, df_career_2013])
player_career_grp = pd.concat([player_career_grp, df_career_2014])
player_career_grp = pd.concat([player_career_grp, df_career_2015])
player_career_grp = pd.concat([player_career_grp, df_career_2016])
player_career_grp = pd.concat([player_career_grp, df_career_2017])
player_career_grp = pd.concat([player_career_grp, df_career_2018])
player_career_grp.head()



att_car  yds_car  td_car  tar_car  rec_yds_car
Name            games Opponent                                                
Adrian Peterson 1     JAX         17.00    84.00    2.00     1.00         3.00
                2     IND         16.50    72.00    1.00     2.00        11.50
                3     SF          19.33    76.67    0.67     2.67        14.67
                4     DET         19.75    83.00    0.50     3.00        16.00
                5     TEN         19.20    84.00    0.40     3.00        15.80

In [54]:
career_shifted = shift_dict(player_career_grp)
career_shifted.head()

games Opponent  att_car  yds_car  td_car  tar_car  \
Aaron Jones 11764    0.0        0      0.0      0.0    0.00     0.00   
            11765    1.0      WAS      6.0     42.0    0.00     1.00   
            11766    2.0      BUF      8.5     53.5    0.50     1.00   
            11767    3.0      DET      8.0     49.0    0.33     1.67   
            11768    4.0       SF      8.0     47.0    0.25     1.50   

                   rec_yds_car  
Aaron Jones 11764         0.00  
            11765         5.00  
            11766        11.00  
            11767        13.67  
            11768        10.25

In [55]:
temp.head()

,year,Name,games,Opponent,att_avg,yds_avg,td_avg,tar_avg,rec_yds_avg,att_ma,yds_ma,td_ma,tar_ma,rec_yds_ma
0,2018,Aaron Jones,1,WAS,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00
1,2018,Aaron Jones,2,BUF,6.0,42.0,0.00,1.00,5.00,0.0,0.0,0.00,0.0,0.00
2,2018,Aaron Jones,3,DET,8.5,53.5,0.50,1.00,11.00,0.0,0.0,0.00,0.0,0.00
3,2018,Aaron Jones,4,SF,8.0,49.0,0.33,1.67,13.67,0.0,0.0,0.00,0.0,0.00
4,2018,Aaron Jones,5,LAR,8.0,47.0,0.25,1.50,10.25,8.0,47.0,0.25,1.5,10.25


In [56]:
career_shifted.reset_index(inplace=True)
career_shifted.head()


,level_0,level_1,games,Opponent,att_car,yds_car,td_car,tar_car,rec_yds_car
0,Aaron Jones,11764,0.0,0,0.0,0.0,0.00,0.00,0.00
1,Aaron Jones,11765,1.0,WAS,6.0,42.0,0.00,1.00,5.00
2,Aaron Jones,11766,2.0,BUF,8.5,53.5,0.50,1.00,11.00
3,Aaron Jones,11767,3.0,DET,8.0,49.0,0.33,1.67,13.67
4,Aaron Jones,11768,4.0,SF,8.0,47.0,0.25,1.50,10.25


In [58]:
career_shifted.drop(['level_0','level_1','games','Opponent'], axis =1, inplace=True)
career_shifted.head()

,att_car,yds_car,td_car,tar_car,rec_yds_car
0,0.0,0.0,0.00,0.00,0.00
1,6.0,42.0,0.00,1.00,5.00
2,8.5,53.5,0.50,1.00,11.00
3,8.0,49.0,0.33,1.67,13.67
4,8.0,47.0,0.25,1.50,10.25


In [169]:
career_shifted.columns

Index(['tar_car', 'rec_car', 'yds_car', 'yds/rec_car', 'yds/tar_car', 'td_car',
       'def_rk_car', 'h/a_1_car', 'surface_0_car', 'points_car'],
      dtype='object')

In [96]:
career_shifted.drop(['points_car','surface_0_car','h/a_1_car','def_rk_car'],axis=1, inplace=True)
career_shifted.head()

,att_car,comp%_car,yds_car,td_car
0,0.00,0.0,0.00,0.0
1,0.00,0.0,0.00,0.0
2,44.00,68.0,303.00,2.0
3,38.00,68.5,261.00,1.5
4,38.33,68.0,248.33,1.0


In [153]:
career_shifted = career_shifted.sort_values(['level_0','games_car']).drop(['level_0','level_1','Week','Opponent'], axis=1)
career_shifted.head()

,games_car,att_car,comp%_car,yds_car,td_car,def_rk_car
0,0.0,0.00,0.0,0.00,0.0,0.00
1,0.0,0.00,0.0,0.00,0.0,0.00
2,1.0,44.00,68.0,303.00,2.0,5.00
3,1.5,38.00,68.5,261.00,1.5,18.00
4,2.0,38.33,68.0,248.33,1.0,12.33


In [59]:
temp = pd.concat([temp, career_shifted], axis=1)
temp.isnull().sum()

year           0
Name           0
games          0
Opponent       0
att_avg        0
yds_avg        0
td_avg         0
tar_avg        0
rec_yds_avg    0
att_ma         0
yds_ma         0
td_ma          0
tar_ma         0
rec_yds_ma     0
att_car        0
yds_car        0
td_car         0
tar_car        0
rec_yds_car    0
dtype: int64

In [60]:
temp.head()

,year,Name,games,Opponent,att_avg,yds_avg,td_avg,tar_avg,rec_yds_avg,att_ma,yds_ma,td_ma,tar_ma,rec_yds_ma,att_car,yds_car,td_car,tar_car,rec_yds_car
0,2018,Aaron Jones,1,WAS,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00
1,2018,Aaron Jones,2,BUF,6.0,42.0,0.00,1.00,5.00,0.0,0.0,0.00,0.0,0.00,6.0,42.0,0.00,1.00,5.00
2,2018,Aaron Jones,3,DET,8.5,53.5,0.50,1.00,11.00,0.0,0.0,0.00,0.0,0.00,8.5,53.5,0.50,1.00,11.00
3,2018,Aaron Jones,4,SF,8.0,49.0,0.33,1.67,13.67,0.0,0.0,0.00,0.0,0.00,8.0,49.0,0.33,1.67,13.67
4,2018,Aaron Jones,5,LAR,8.0,47.0,0.25,1.50,10.25,8.0,47.0,0.25,1.5,10.25,8.0,47.0,0.25,1.50,10.25


In [61]:
rb.head()

,games,year,Name,Team,Week,Opponent,att,yds,td,tar,rec_yds,def_rk,h/a_1,surface_0,points
0,1,2018,Aaron Jones,GB,3,WAS,6,42,0,1,5,15,0.0,1.0,5.7
1,2,2018,Aaron Jones,GB,4,BUF,11,65,1,1,17,18,1.0,1.0,15.2
2,3,2018,Aaron Jones,GB,5,DET,7,40,0,3,19,29,0.0,0.0,7.9
3,4,2018,Aaron Jones,GB,6,SF,8,41,0,1,0,22,0.0,1.0,4.1
4,5,2018,Aaron Jones,GB,8,LAR,12,86,1,2,0,14,0.0,1.0,16.6


In [62]:
temp['surface_0']=rb['surface_0'].values
temp['h/a_1']=rb['h/a_1'].values
temp['def_rk'] = rb['def_rk']
temp['points'] = rb['points']

In [63]:
temp.head()

,year,Name,games,Opponent,att_avg,yds_avg,td_avg,tar_avg,rec_yds_avg,att_ma,...,rec_yds_ma,att_car,yds_car,td_car,tar_car,rec_yds_car,surface_0,h/a_1,def_rk,points
0,2018,Aaron Jones,1,WAS,0.0,0.0,0.00,0.00,0.00,0.0,...,0.00,0.0,0.0,0.00,0.00,0.00,1.0,0.0,15,5.7
1,2018,Aaron Jones,2,BUF,6.0,42.0,0.00,1.00,5.00,0.0,...,0.00,6.0,42.0,0.00,1.00,5.00,1.0,1.0,18,15.2
2,2018,Aaron Jones,3,DET,8.5,53.5,0.50,1.00,11.00,0.0,...,0.00,8.5,53.5,0.50,1.00,11.00,0.0,0.0,29,7.9
3,2018,Aaron Jones,4,SF,8.0,49.0,0.33,1.67,13.67,0.0,...,0.00,8.0,49.0,0.33,1.67,13.67,1.0,0.0,22,4.1
4,2018,Aaron Jones,5,LAR,8.0,47.0,0.25,1.50,10.25,8.0,...,10.25,8.0,47.0,0.25,1.50,10.25,1.0,0.0,14,16.6


In [64]:
rb_stats = temp
rb_stats.shape

(12612, 23)

In [65]:
#A function that will covert a time series database into a supervised learning database

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
            cols.append(df.shift(i))
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [160]:
# A function to insert the non reframed columns back into the stats dataframe 

def insert_non_reframed(df, col_dict):
    non_reframed = ['games','year','Opponent','def_rk','h/a_1','surface_0','points']
    for player in df['Name'].unique():
        for col in non_reframed:
            reframed_dict['{}_reframed'.format(player)][col] = col_dict['{}'.format(player)][col].values
            
    return reframed_dict

In [65]:
rb_stats.head()

,year,Name,games,Opponent,att_avg,yds_avg,td_avg,tar_avg,rec_yds_avg,att_ma,...,rec_yds_ma,att_car,yds_car,td_car,tar_car,rec_yds_car,surface_0,h/a_1,def_rk,points
0,2018,Aaron Jones,1,WAS,0.0,0.0,0.00,0.00,0.00,0.0,...,0.00,0.0,0.0,0.00,0.00,0.00,1.0,0.0,15,5.7
1,2018,Aaron Jones,2,BUF,6.0,42.0,0.00,1.00,5.00,0.0,...,0.00,6.0,42.0,0.00,1.00,5.00,1.0,1.0,18,15.2
2,2018,Aaron Jones,3,DET,8.5,53.5,0.50,1.00,11.00,0.0,...,0.00,8.5,53.5,0.50,1.00,11.00,0.0,0.0,29,7.9
3,2018,Aaron Jones,4,SF,8.0,49.0,0.33,1.67,13.67,0.0,...,0.00,8.0,49.0,0.33,1.67,13.67,1.0,0.0,22,4.1
4,2018,Aaron Jones,5,LAR,8.0,47.0,0.25,1.50,10.25,8.0,...,10.25,8.0,47.0,0.25,1.50,10.25,1.0,0.0,14,16.6


In [67]:
rb_stats.to_csv('input/rb/rb_stats.csv', index=False)

In [68]:
rb_stats= pd.read_csv('input/rb/rb_stats.csv')


In [69]:
rb_stats.shape

(12612, 23)

In [70]:
rb_stats.isnull().sum()

year           0
Name           0
games          0
Opponent       0
att_avg        0
yds_avg        0
td_avg         0
tar_avg        0
rec_yds_avg    0
att_ma         0
yds_ma         0
td_ma          0
tar_ma         0
rec_yds_ma     0
att_car        0
yds_car        0
td_car         0
tar_car        0
rec_yds_car    0
surface_0      0
h/a_1          0
def_rk         0
points         0
dtype: int64